<a href="https://colab.research.google.com/github/wxbaum/twitter_influencers_analysis/blob/master/twitter_infuencers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tweepy
import time

In [0]:
"""
These keys/tokes require a Twitter developer account. Sign up for a developer 
account at https://developer.twitter.com and create a new app.
""" 

api_key = 'aoGosXnGsr6KM1VHpISICxRPI'
secret_key = 'fEyygkhWQK6aaIHqONLdPhhRs4hvqrgyZu3OuWE6PCo2bUXQus'
access_token = '141198705-IlSTJwpgWm5PrOPbpO35pRJv5OxIAAu8b67Alo9j'
secret_token = 'd4yu7qhV80Wn2gXKDHuLJ3mb1osEYH88ck91mIWbkBC9a'

def authenticate(api_key, secret_key, access_token, secret_token):

  auth = tweepy.OAuthHandler(api_key, secret_key)
  auth.set_access_token(access_token, secret_token)
  api = tweepy.API(auth)
  
  return api


api = authenticate(api_key, secret_key, access_token, secret_token)

In [0]:
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(5)

In [57]:
# Adding actual functionality 

Follower list count: 450
